In [111]:
import pydicom
import glob
import os
import natsort
import png
import numpy as np
import cv2
from cv2 import dnn_superres
dossier_input = "input"
dossier_output_png = "output_png"
dossier_output_png_up = "output_png_up"
dossier_output_video = "output_video"
dossier_output_video_up = "output_video_up"
skip_frames = 8

In [ ]:
for base, folders, files  in os.walk(dossier_input):
    overall_max = 0
    i = 0
    for file in natsort.natsorted(files):
        try:
            ds = pydicom.dcmread(os.path.join(base,file))
            data_type = type(ds.PixelData)
            if ds.LargestImagePixelValue > overall_max and i > skip_frames:
                overall_max = ds.LargestImagePixelValue
            if i == 0:
                print(ds.Rows, ds.Columns, ds.PixelSpacing)
            i += 1
        except:
            continue
    for file in natsort.natsorted(files):
        try:
            ds = pydicom.dcmread(os.path.join(base,file))
            data_type = type(ds.PixelData)
            shape = ds.pixel_array.shape
            image_2d = ds.pixel_array.astype(float)
            image_2d_scaled = (image_2d / overall_max) * 255.0
            out_target = base.replace(dossier_input, dossier_output_png)
            if not os.path.exists(out_target):
                os.makedirs(out_target)
            
            with open(os.path.join(out_target,"img"+f'{ds.TemporalPositionIndex:05}'+'.png') , 'wb') as png_file:
                w = png.Writer(shape[1], shape[0], greyscale=True)
                w.write(png_file, image_2d_scaled.astype(np.uint8))
        except:
            continue

In [119]:
for base, folders, files  in os.walk(dossier_output_png):
    files = natsort.natsorted(files)
    i = 0
    out_target = base.replace(dossier_output_png, dossier_output_video)
    if len(files) > 1:
        if not os.path.exists(os.path.dirname(out_target)):
            os.makedirs(os.path.dirname(out_target))
        frame = cv2.imread(os.path.join(base, files[0]))
        height, width, layers = frame.shape
        video = cv2.VideoWriter(out_target+".avi", 0, 10, (width,height))
        for image in files:
            video.write(cv2.imread(os.path.join(base, image)))
        cv2.destroyAllWindows()
        video.release()
        

In [122]:
# Create an SR object
sr = dnn_superres.DnnSuperResImpl_create()
# Read the desired model
path = "FSRCNN_x4.pb"
sr.readModel(path)
# Set the desired model and scale to get correct pre- and post-processing
sr.setModel("fsrcnn", 4)

for base, folders, files  in os.walk(dossier_output_png):
    i = 0
    for file in natsort.natsorted(files):
        out_target = base.replace(dossier_output_png, dossier_output_png_up)
        if not os.path.exists(out_target):
            os.makedirs(out_target)
            # Read image
        image = cv2.imread(os.path.join(base,file))
        # Upscale the image
        result = sr.upsample(image)
        # Save the image
        cv2.imwrite(os.path.join(out_target,file), result)

In [109]:
for base, folders, files  in os.walk(dossier_output_png_up):
    files = natsort.natsorted(files)
    i = 0
    out_target = base.replace(dossier_output_png_up, dossier_output_video_up)
    if not os.path.exists(os.path.dirname(out_target)):
        os.makedirs(os.path.dirname(out_target))
    if len(files) > 1:
        frame = cv2.imread(os.path.join(base, files[0]))
        height, width, layers = frame.shape
        video = cv2.VideoWriter(out_target+".avi", 0, 10, (width,height))
        for image in files:
            video.write(cv2.imread(os.path.join(base, image)))
        cv2.destroyAllWindows()
        video.release()